In [1]:
# ==============================================================================
# SCRIPT D'ENTRAÎNEMENT FINAL - VERSION DÉFINITIVE ET CORRIGÉE
# ==============================================================================

# --- 1. SETUP AND IMPORTS ---
import os
import gc
import tensorflow as tf
from tensorflow.keras import layers, models, applications, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.optimizers.schedules import CosineDecay
import matplotlib.pyplot as plt

print(f"TensorFlow Version: {tf.__version__}")

# --- 2. DATASET DOWNLOAD FROM GOOGLE DRIVE ---
!pip install -q gdown
gdrive_link = "https://drive.google.com/file/d/1oaJcF-Oe-81OD9wp16VExOJgLuc8vU94/view?usp=drive_link"
output_zip_path = "/kaggle/working/dataset.zip"
extract_path = "/kaggle/working/datasets/"
print("📂 Téléchargement du dataset...")
!gdown --fuzzy "{gdrive_link}" -O "{output_zip_path}"
print("📦 Décompression du dataset...")
!unzip -q -o "{output_zip_path}" -d "{extract_path}"
print(f"✅ Dataset prêt dans {extract_path}")

# --- 3. CONFIGURATION ---
IMG_SIZE = 224
BATCH_SIZE = 24
EPOCHS_PHASE_1 = 20
EPOCHS_PHASE_2 = 40
AUTOTUNE = tf.data.AUTOTUNE

TRAIN_DIR = os.path.join(extract_path, "train")
TEST_DIR = os.path.join(extract_path, "test")
CHECKPOINT_DIR = "/kaggle/working/AI_Checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
CHECKPOINT_PATH = os.path.join(CHECKPOINT_DIR, "emotion_efficientnetv2M_SOTA.weights.h5")

tf.keras.mixed_precision.set_global_policy('mixed_float16')
print("\n✅ Mixed Precision Training Enabled.")
print(f"📐 Image Resolution: {IMG_SIZE}x{IMG_SIZE}")

# --- 4. DATA PREPARATION & AUGMENTATION ---
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"), layers.RandomRotation(0.15),
    layers.RandomZoom(0.15), layers.RandomTranslation(0.1, 0.1),
    layers.RandomContrast(0.15),
], name="data_augmentation")

def cutout(images, labels):
    image_height = tf.shape(images)[1]
    image_width = tf.shape(images)[2]
    batch_size = tf.shape(images)[0]
    scale=(0.02, 0.1); ratio=(0.3, 3.3)
    area = tf.cast(image_height * image_width, tf.float32)
    erase_area = tf.random.uniform([batch_size], scale[0], scale[1]) * area
    aspect_ratio = tf.random.uniform([batch_size], ratio[0], ratio[1])
    h = tf.cast(tf.round(tf.sqrt(erase_area * aspect_ratio)), tf.int32)
    w = tf.cast(tf.round(tf.sqrt(erase_area / aspect_ratio)), tf.int32)
    h = tf.minimum(h, image_height); w = tf.minimum(w, image_width)
    x_rate = tf.random.uniform([batch_size]); y_rate = tf.random.uniform([batch_size])
    x = tf.cast(x_rate * tf.cast(image_width - w, tf.float32), tf.int32)
    y = tf.cast(y_rate * tf.cast(image_height - h, tf.float32), tf.int32)

    # --- CORRECTION DE BROADCASTING ---
    # Créer les grilles de coordonnées pour toutes les images
    row_coords = tf.cast(tf.range(image_height), dtype=tf.int32)
    col_coords = tf.cast(tf.range(image_width), dtype=tf.int32)
    
    # Adapter les formes pour le broadcasting (la forme correcte est (1, H, 1, 1) et (1, 1, W, 1))
    row_coords = tf.reshape(row_coords, (1, image_height, 1, 1))
    col_coords = tf.reshape(col_coords, (1, 1, image_width, 1))
    # --- FIN DE LA CORRECTION ---

    y_b = tf.reshape(y, (batch_size, 1, 1, 1)); h_b = tf.reshape(h, (batch_size, 1, 1, 1))
    x_b = tf.reshape(x, (batch_size, 1, 1, 1)); w_b = tf.reshape(w, (batch_size, 1, 1, 1))
    
    mask_y = tf.logical_and(row_coords >= y_b, row_coords < y_b + h_b)
    mask_x = tf.logical_and(col_coords >= x_b, col_coords < x_b + w_b)
    mask = tf.cast(tf.logical_and(mask_y, mask_x), images.dtype)
    return images * (1.0 - mask), labels

def create_dataset(directory, augment=False):
    initial_ds = tf.keras.utils.image_dataset_from_directory(
        directory, label_mode='categorical', image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE, shuffle=True if augment else False
    )
    class_names = initial_ds.class_names
    dataset = initial_ds.map(lambda x, y: (tf.cast(x, tf.float32), y), num_parallel_calls=AUTOTUNE)
    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
        dataset = dataset.map(cutout, num_parallel_calls=AUTOTUNE)
    return dataset.prefetch(buffer_size=AUTOTUNE), class_names

print("\nLoading and preparing datasets...")
train_dataset, class_names = create_dataset(TRAIN_DIR, augment=True)
test_dataset, _ = create_dataset(TEST_DIR, augment=False)
NUM_CLASSES = len(class_names)
print(f"✅ Found {NUM_CLASSES} classes: {class_names}")

# --- 5. MODEL DEFINITION ---
inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
base_model = applications.EfficientNetV2M(
    input_shape=(IMG_SIZE, IMG_SIZE, 3), include_top=False, weights="imagenet"
)
base_model.trainable = False
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(768, activation="gelu", kernel_regularizer=regularizers.l2(1e-4))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(NUM_CLASSES, activation="softmax", dtype=tf.float32)(x)
model = models.Model(inputs, outputs)

print("\nBuilding SOTA model with EfficientNetV2-M base...")
model.summary()

# --- 6. ADVANCED TRAINING STRATEGY ---
@tf.keras.utils.register_keras_serializable()
class FocalLoss(tf.keras.losses.Loss):
    def __init__(self, gamma=2.0, alpha=0.25, name='focal_loss'):
        super().__init__(name=name)
        self.gamma = gamma; self.alpha = alpha
    def call(self, y_true, y_pred):
        ce = tf.keras.losses.categorical_crossentropy(y_true, y_pred, from_logits=False)
        p_t = tf.reduce_sum(y_true * y_pred, axis=-1)
        loss = self.alpha * tf.pow(1.0 - p_t, self.gamma) * ce
        return tf.reduce_mean(loss)

# --- PHASE 1 ---
print("\n" + "="*50); print("🚀 PHASE 1: Training the SOTA Head"); print("="*50)
total_steps_phase1 = len(train_dataset) * EPOCHS_PHASE_1
lr_schedule_phase1 = CosineDecay(initial_learning_rate=1e-3, decay_steps=total_steps_phase1, alpha=0.01)
callbacks_phase1 = [
    EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True, verbose=1),
    ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]
model.compile(optimizer=AdamW(learning_rate=lr_schedule_phase1, weight_decay=1e-4), loss=FocalLoss(), metrics=["accuracy"])
history_phase1 = model.fit(train_dataset, validation_data=test_dataset, epochs=EPOCHS_PHASE_1, callbacks=callbacks_phase1)

# --- PHASE 2 ---
print("\n" + "="*50); print("🔧 PHASE 2: Full Network Fine-Tuning"); print("="*50)
base_model.trainable = True
fine_tune_at = int(len(base_model.layers) * 0.50)
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False
for layer in base_model.layers:
    if isinstance(layer, layers.BatchNormalization): layer.trainable = False
print(f"🔓 Unfrozen {len(base_model.layers) - fine_tune_at} layers.")
total_steps_phase2 = len(train_dataset) * EPOCHS_PHASE_2
lr_schedule_phase2 = CosineDecay(initial_learning_rate=1e-5, decay_steps=total_steps_phase2, alpha=0.1)
callbacks_phase2 = [
    EarlyStopping(monitor='val_accuracy', patience=12, restore_best_weights=True, verbose=1),
    ModelCheckpoint(filepath=CHECKPOINT_PATH, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)
]
model.compile(optimizer=AdamW(learning_rate=lr_schedule_phase2, weight_decay=1e-5), loss=FocalLoss(), metrics=["accuracy"])
history_phase2 = model.fit(train_dataset, validation_data=test_dataset,
    epochs=EPOCHS_PHASE_2,
    initial_epoch=history_phase1.epoch[-1] if history_phase1.epoch and len(history_phase1.epoch) > 0 else 0,
    callbacks=callbacks_phase2)

# --- 7. FINAL EVALUATION AND SAVING (CORRECTED & SIMPLIFIED) ---
print("\n" + "="*50); print("📊 FINAL EVALUATION"); print("="*50)
print("✅ Best model weights are already in memory from training.")
test_loss, test_acc = model.evaluate(test_dataset, verbose=1)
print(f"\n🎯 Final Test Accuracy: {test_acc*100:.2f}%")

final_model_name = f"sota_emotion_model_final_acc_{test_acc*100:.2f}.keras"
final_model_path = os.path.join(CHECKPOINT_DIR, final_model_name)
model.save(final_model_path)
print(f"💾 Model saved to: {final_model_path}")
print("\n🎉 ENTRAÎNEMENT TERMINÉ ET MODÈLE SAUVEGARDÉ AVEC SUCCÈS !")
gc.collect()

2025-08-23 18:49:03.714260: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755974944.086553      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755974944.199825      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


TensorFlow Version: 2.18.0
📂 Téléchargement du dataset...
Downloading...
From (original): https://drive.google.com/uc?id=1oaJcF-Oe-81OD9wp16VExOJgLuc8vU94
From (redirected): https://drive.google.com/uc?id=1oaJcF-Oe-81OD9wp16VExOJgLuc8vU94&confirm=t&uuid=49e5c91e-439a-4111-9e06-dd5de525de48
To: /kaggle/working/dataset.zip
100%|████████████████████████████████████████| 117M/117M [00:01<00:00, 92.3MB/s]
📦 Décompression du dataset...
✅ Dataset prêt dans /kaggle/working/datasets/

✅ Mixed Precision Training Enabled.
📐 Image Resolution: 224x224


I0000 00:00:1755974977.055593      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1755974977.056514      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



Loading and preparing datasets...
Found 41882 files belonging to 7 classes.
Found 10246 files belonging to 7 classes.
✅ Found 7 classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step

Building SOTA model with EfficientNetV2-M base...


Model: "functional_1"

...

I0000 00:00:1755975048.976609     110 service.cc:148] XLA service 0x7b26d40f1b20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1755975048.978010     110 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1755975048.978034     110 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1755975056.104771     110 cuda_dnn.cc:529] Loaded cuDNN version 90300


   1/1746 ━━━━━━━━━━━━━━━━━━━━ 50:29:30 104s/step - accuracy: 0.0417 - loss: 1.0191

I0000 00:00:1755975093.318191     110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1746/1746 ━━━━━━━━━━━━━━━━━━━━ 0s 240ms/step - accuracy: 0.3448 - loss: 0.5313
Epoch 1: val_accuracy improved from -inf to 0.45159, saving model to /kaggle/working/AI_Checkpoints/emotion_efficientnetv2M_SOTA.weights.h5
1746/1746 ━━━━━━━━━━━━━━━━━━━━ 580s 273ms/step - accuracy: 0.3448 - loss: 0.5313 - val_accuracy: 0.4516 - val_loss: 0.3415
Epoch 2/20
1745/1746 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.4011 - loss: 0.3473
Epoch 2: val_accuracy improved from 0.45159 to 0.48048, saving model to /kaggle/working/AI_Checkpoints/emotion_efficientnetv2M_SOTA.weights.h5
1746/1746 ━━━━━━━━━━━━━━━━━━━━ 409s 233ms/step - accuracy: 0.4011 - loss: 0.3473 - val_accuracy: 0.4805 - val_loss: 0.2981
Epoch 3/20
1745/1746 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step - accuracy: 0.4093 - loss: 0.3214
Epoch 3: val_accuracy did not improve from 0.48048
1746/1746 ━━━━━━━━━━━━━━━━━━━━ 440s 232ms/step - accuracy: 0.4093 - loss: 0.3214 - val_accuracy: 0.4786 - val_loss: 0.2900
Epoch 4/20
1745/1746 ━━━━━━━━━━━━━━━━━

914